In [4]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [5]:
from BertModel import WeightedBertForSequenceClassification


In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [7]:
model_path = '../torch_tcav/model_train_scripts/checkpoints/FINALWeightedBlueBert512NicuLosAdmissions_2/checkpoint-26015'
bert = WeightedBertForSequenceClassification.from_pretrained(model_path,output_attentions=True)

In [22]:
test_df = pd.read_csv('../tcav2/data/los_nicu_test_admissions.csv')


In [23]:
test_df

,Unnamed: 0,new_HADM,Unnamed: 0.1,TEXT,label,ROW_ID,SUBJECT_ID,CHARTTIME,AGE,GENDER
0,0,104132,411459,Neonatology Attending Triage Note\n\nAsked by ...,0,2051008,28711,2100-07-02 23:23:00,0.878472,M
1,1,104551,262961,Neonatology Attending\n\nPreterm Infant with h...,0,1928397,17885,2100-08-06 15:49:00,0.598611,F
2,2,173205,404983,Nursing NICU Note\n\n\nPt. admitted to NICU fo...,0,1796863,8460,2100-10-11 15:10:00,0.516667,M
3,3,138538,97070,Neonatology Attending Admission Note\n\nInfant...,1,1693481,1022,2100-11-10 21:27:00,0.872917,M
4,4,110888,405994,Neonatology Attending Triage Note\n\nAsked by ...,0,1868767,13358,2100-11-20 00:19:00,0.934722,F
...,...,...,...,...,...,...,...,...,...,...
1886,1886,171734,406706,NICU NURSING ADMISSION NOTE\n\n Baby [**Name ...,0,1862578,13025,2201-03-31 18:45:00,0.725000,M
1887,1887,109331,405649,Neonatology-NNP Triage Note\n\nRequested by Dr...,0,1838606,11400,2201-06-25 01:54:00,0.004861,M
1888,1888,118562,410860,Neonatology NP Admission Note\nThis 3.720 kg m...,0,2004464,24254,2201-06-30 05:03:00,0.764583,M
1889,1889,143841,343728,Neonatology Attending\n38 week GA infant admit...,0,1867083,13243,2201-07-16 20:49:00,0.598611,M


In [ ]:
rows = []
for row in test_df.iterrows():
    txt=row[1]['TEXT']
    label=row[1]['label']

    tokens = torch.LongTensor([tokenizer.encode(txt, truncation=True, max_length=512)])

    out = bert(input_ids = tokens, return_logits=False, return_dict=True)

    prob = torch.softmax(out['logits'], 1 ).detach().numpy()[0][1]

    attentions = out['attentions']

    attns = np.array([x.detach().numpy()[0] for x in attentions]).mean(0).mean(0).mean(0)
    rows.append([txt,label,tokens,prob,attns])

In [ ]:
nicu_attentions_df = pd.DataFrame(rows, columns=['TEXT','label', 'ids','prob','attention'])

In [ ]:
nicu_attentions_df['tokens']=nicu_attentions_df['TEXT'].apply(lambda x: list(tokenizer.encode(x, truncation=True, max_length=512)))

In [ ]:
nicu_attentions_df['attention'] = nicu_attentions_df['attention'].apply(lambda x: list(x) )

In [21]:
nicu_attentions_df['ids']=nicu_attentions_df['ids'].apply(lambda x: [int(y.detach()) for y in x[0]])

In [22]:
nicu_attentions_df.to_csv('nicu_attentions.csv')

In [24]:
nicu_attentions_df = pd.read_csv('nicu_attentions.csv')

In [25]:
nicu_attentions_df['tokens'] = nicu_attentions_df['tokens'].apply(eval)

In [26]:
nicu_attentions_df['tokens'] = nicu_attentions_df['tokens'].apply(lambda x: [tokenizer.decode([y]) for y in x])

In [27]:
nicu_attentions_df

,Unnamed: 0,TEXT,label,ids,prob,attention,tokens
0,0,Neonatology Attending Triage Note\n\nAsked by ...,0,"[101, 16231, 10610, 6483, 7052, 13012, 4270, 3...",8.522414e-07,"[0.02902568, 0.0008587185, 0.00095757225, 0.00...","[[CLS], neon, ##ato, ##logy, attending, tri, #..."
1,1,Neonatology Attending\n\nPreterm Infant with h...,0,"[101, 16231, 10610, 6483, 7052, 3653, 3334, 22...",1.620253e-05,"[0.033136137, 0.0010808081, 0.001066551, 0.000...","[[CLS], neon, ##ato, ##logy, attending, pre, #..."
2,2,Nursing NICU Note\n\n\nPt. admitted to NICU fo...,0,"[101, 8329, 27969, 2226, 3602, 13866, 1012, 49...",8.333849e-07,"[0.084175155, 0.012545219, 0.008986112, 0.0072...","[[CLS], nursing, nic, ##u, note, pt, ., admitt..."
3,3,Neonatology Attending Admission Note\n\nInfant...,1,"[101, 16231, 10610, 6483, 7052, 9634, 3602, 10...",9.999992e-01,"[0.030207794, 0.0009799687, 0.0009277218, 0.00...","[[CLS], neon, ##ato, ##logy, attending, admiss..."
4,4,Neonatology Attending Triage Note\n\nAsked by ...,0,"[101, 16231, 10610, 6483, 7052, 13012, 4270, 3...",8.581985e-07,"[0.031253558, 0.00095942826, 0.0011693386, 0.0...","[[CLS], neon, ##ato, ##logy, attending, tri, #..."
...,...,...,...,...,...,...,...
1886,1886,NICU NURSING ADMISSION NOTE\n\n Baby [**Name ...,0,"[101, 27969, 2226, 8329, 9634, 3602, 3336, 103...",1.003571e-06,"[0.039997324, 0.0017908189, 0.0016341269, 0.00...","[[CLS], nic, ##u, nursing, admission, note, ba..."
1887,1887,Neonatology-NNP Triage Note\n\nRequested by Dr...,0,"[101, 16231, 10610, 6483, 1011, 1050, 16275, 1...",9.394774e-07,"[0.034236155, 0.0012748321, 0.0014094624, 0.00...","[[CLS], neon, ##ato, ##logy, -, n, ##np, tri, ..."
1888,1888,Neonatology NP Admission Note\nThis 3.720 kg m...,0,"[101, 16231, 10610, 6483, 27937, 9634, 3602, 2...",8.765282e-07,"[0.02903299, 0.0009647377, 0.0010617776, 0.001...","[[CLS], neon, ##ato, ##logy, np, admission, no..."
1889,1889,Neonatology Attending\n38 week GA infant admit...,0,"[101, 16231, 10610, 6483, 7052, 4229, 2733, 11...",1.075447e-06,"[0.026149474, 0.0008155299, 0.00090316724, 0.0...","[[CLS], neon, ##ato, ##logy, attending, 38, we..."


In [28]:
nicu_attentions_df['attention'] = nicu_attentions_df['attention'].apply(eval)

In [29]:
nicu_attentions_df

,Unnamed: 0,TEXT,label,ids,prob,attention,tokens
0,0,Neonatology Attending Triage Note\n\nAsked by ...,0,"[101, 16231, 10610, 6483, 7052, 13012, 4270, 3...",8.522414e-07,"[0.02902568, 0.0008587185, 0.00095757225, 0.00...","[[CLS], neon, ##ato, ##logy, attending, tri, #..."
1,1,Neonatology Attending\n\nPreterm Infant with h...,0,"[101, 16231, 10610, 6483, 7052, 3653, 3334, 22...",1.620253e-05,"[0.033136137, 0.0010808081, 0.001066551, 0.000...","[[CLS], neon, ##ato, ##logy, attending, pre, #..."
2,2,Nursing NICU Note\n\n\nPt. admitted to NICU fo...,0,"[101, 8329, 27969, 2226, 3602, 13866, 1012, 49...",8.333849e-07,"[0.084175155, 0.012545219, 0.008986112, 0.0072...","[[CLS], nursing, nic, ##u, note, pt, ., admitt..."
3,3,Neonatology Attending Admission Note\n\nInfant...,1,"[101, 16231, 10610, 6483, 7052, 9634, 3602, 10...",9.999992e-01,"[0.030207794, 0.0009799687, 0.0009277218, 0.00...","[[CLS], neon, ##ato, ##logy, attending, admiss..."
4,4,Neonatology Attending Triage Note\n\nAsked by ...,0,"[101, 16231, 10610, 6483, 7052, 13012, 4270, 3...",8.581985e-07,"[0.031253558, 0.00095942826, 0.0011693386, 0.0...","[[CLS], neon, ##ato, ##logy, attending, tri, #..."
...,...,...,...,...,...,...,...
1886,1886,NICU NURSING ADMISSION NOTE\n\n Baby [**Name ...,0,"[101, 27969, 2226, 8329, 9634, 3602, 3336, 103...",1.003571e-06,"[0.039997324, 0.0017908189, 0.0016341269, 0.00...","[[CLS], nic, ##u, nursing, admission, note, ba..."
1887,1887,Neonatology-NNP Triage Note\n\nRequested by Dr...,0,"[101, 16231, 10610, 6483, 1011, 1050, 16275, 1...",9.394774e-07,"[0.034236155, 0.0012748321, 0.0014094624, 0.00...","[[CLS], neon, ##ato, ##logy, -, n, ##np, tri, ..."
1888,1888,Neonatology NP Admission Note\nThis 3.720 kg m...,0,"[101, 16231, 10610, 6483, 27937, 9634, 3602, 2...",8.765282e-07,"[0.02903299, 0.0009647377, 0.0010617776, 0.001...","[[CLS], neon, ##ato, ##logy, np, admission, no..."
1889,1889,Neonatology Attending\n38 week GA infant admit...,0,"[101, 16231, 10610, 6483, 7052, 4229, 2733, 11...",1.075447e-06,"[0.026149474, 0.0008155299, 0.00090316724, 0.0...","[[CLS], neon, ##ato, ##logy, attending, 38, we..."


In [30]:
def unravel(lst):
    return [i for j in lst for i in j]

In [31]:
# label = 1
all_class_attentions = pd.DataFrame(  list(zip( unravel(nicu_attentions_df['tokens']),  unravel(nicu_attentions_df['attention']) )) , columns=['tokens', 'attention']  )

In [32]:
all_class_attentions

,tokens,attention
0,[CLS],0.029026
1,neon,0.000859
2,##ato,0.000958
3,##logy,0.000890
4,attending,0.001523
...,...,...
686126,for,0.002725
686127,further,0.003120
686128,details,0.003919
686129,.,0.004738


In [33]:
word =''
score=0
words=[]
scores=[]
for i in reversed(range(len(all_class_attentions))):
    token=all_class_attentions['tokens'].iloc[i]
    if token.startswith('##'):
        word = token[2:]+word
        score = max(score,all_class_attentions['attention'].iloc[i] )
#         print(word)
    else:
        word = token+word
#         print(word)
        words.append(word)
        score = max(score,all_class_attentions['attention'].iloc[i] )
        scores.append(score)
        word=''
        score=0

In [35]:
tokens_grouped = pd.DataFrame(list(zip(words, scores)), columns=['words', 'attention']).groupby('words')
means = tokens_grouped['attention'].mean()
cts = tokens_grouped['attention'].count()
wrds = tokens_grouped.apply(lambda group: str(group.name))
class_df = pd.DataFrame(list(zip(wrds,means,cts)), columns=['word','attention_mean','word_count' ]).sort_values(by='attention_mean',ascending=False)
class_df = class_df[class_df['word_count']>5]
print(len(class_df))
# print(class_df.head(25).to_latex(index=False, float_format="%.3f"))
class_df.head(27)

3074


,word,attention_mean,word_count
3422,[SEP],0.169954,1891
3421,[CLS],0.036695,1891
11321,trace,0.036231,9
7827,lsc,0.034579,9
9161,picture,0.034450,6
7603,labetolol,0.031846,14
5748,dtv,0.031454,7
8255,murmer,0.029267,15
1652,2935,0.029120,6
6351,flovent,0.027581,6


In [38]:
tokenizer.build_inputs_with_special_tokens([])

[101, 102]

In [40]:
tokenizer.encode('')

[101, 102]